In [ ]:
import pandas as pd
import numpy as np
import json
import io
import datetime as dt
import string
import unicodedata

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
# stopword_list = nltk.corpus.stopwords.words('english')

import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
# nlp = spacy.load('en_core', parse=True, tag=True, entity=True)

import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

from gensim import corpora, models, similarities

#nlp_vec = spacy.load('en_vecs', parse = True, tag=True, #entity=True)

In [ ]:
#create dataframe from input data; identify the column with the tweet text
path = '/Users/kavyasuresh/DS/Social-Stocking/Data set/MSFT/'
df=pd.read_csv(path+'MSFT_aggregated.csv', index_col=0)
print('Number of observations are: '+str(len(df)))

In [ ]:
tweet_dictionary = {}
i = 0
for line in df['Tweet']:
        print(line)
        if(type(line) == str):
            tweet_dictionary[i] = line.lower()
        else:
            tweet_dictionary[i] = "" 
        i += 1
print(tweet_dictionary[4])

In [ ]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_links(tweet_dictionary[i])

def strip_mentions(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def strip_symbols(text):
    entity_prefixes = ['$']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_mentions(tweet_dictionary[i])

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_symbols(tweet_dictionary[i])

def strip_hashtags(text):
    entity_prefixes = ['#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_hashtags(tweet_dictionary[i])

for i in range(0,len(df)):
    tweet_dictionary[i] = tweet_dictionary[i].replace('RT', '')

#code source is http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "youyou will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    text = c_re.sub(replace, text.lower())
    return text

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=expandContractions(tweet_dictionary[i])

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=remove_special_characters(tweet_dictionary[i], 
                          remove_digits=True)
# print (tweet_dictionary)

# df_ticker=pd.read_csv('nadsaq_ticker.csv')
# #print(str(len(df_ticker)))
# list1=[]
# for index, row in df_ticker.iterrows():
#     row['Company'] = row['Company'].replace(' ','')
#     list1.append(row['Company'].replace('$','').lower())
# #     list1.append(row['Company'])
# # print(list1)
# # list1.append('$LIT')
# tweet_text = []
# i= 0
# #print(tweet_dictionary)
# # for i in range(0,len(tweet_dictionary)):
# #     temp=[]
# #     word =tweet_dictionary[i] 
# #     if word not in list1:
# # #             print(word)
# #             temp.append(word)
# #     print(tweet_dictionary[i])
# # #     tweet_dictionary[i] = str(' '.join(temp))
# # # #     tweet_text.append(list(temp))
# # #     print(tweet_dictionary)
# for key,val in tweet_dictionary.items():
#     temp = []
#     for word in val.split():
#         if word not in list1:
#             temp.append(word)
#     tweet_dictionary[key] = ' '.join(temp)
# print(tweet_dictionary)
    

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

stopword_list = nltk.corpus.stopwords.words('english')
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=remove_stopwords(tweet_dictionary[i])

def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=simple_stemmer(tweet_dictionary[i])
    #tweet_dictionary[i]=lemmatize_text(tweet_dictionary[i])

#Create corupus of all words
words_corpus = []
for i in range(0,len(tweet_dictionary)):
        words_corpus.append([word for word in tweet_dictionary[i].lower().split() if len(word)>3])

dictionary = corpora.Dictionary(words_corpus)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
# no_below is minimum number of documents
# no_above is a percentage
# keep_n is the thresshold for the max number of words to keep


corpus_bow = [dictionary.doc2bow(text) for text in words_corpus]

from gensim import corpora, models
tfidf = models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]

# sentence = 'London is the capital and most populous city of England and the United Kingdom'

# sentence_nlp = nlp(sentence)

# # POS tagging with Spacy 
# spacy_pos_tagged = [(word, word.tag_, word.pos_) for word in sentence_nlp]
# pd.DataFrame(spacy_pos_tagged, columns=['Word', 'POS tag', 'Tag type'])

# # POS tagging with nltk
# #nltk_pos_tagged = nltk.pos_tag(sentence.split())
# #pd.DataFrame(nltk_pos_tagged, columns=['Word', 'POS tag'])

# ntopics = 5
# lsi = models.LsiModel(corpus_bow, id2word=dictionary, num_topics=ntopics)
# query = sentence
# vec_bow = dictionary.doc2bow(query.lower().split())
# vec_lsi = lsi[vec_bow]
# print(vec_lsi)
# index = similarities.MatrixSimilarity(lsi[corpus_bow])
# sims = index[vec_lsi]

for ki in range(ntopics):
    print("Topic %d: %s\n" % (ki, lsi.print_topic(ki)))

lda_model = models.LdaMulticore(corpus_bow, num_topics=2, id2word=dictionary, passes=2, workers=2)


#####TOPICS######
# Score: 0.8645382523536682	 
# Topic: 0.016*"help" + 0.015*"microsoft" + 0.014*"data" + 0.014*"work" + 0.012*"partner" + 0.011*"make" + 0.011*"product" + 0.011*"window" + 0.010*"compani" + 0.009*"year"

# Score: 0.03449874743819237	 
# Topic: 0.022*"microsoft" + 0.021*"secur" + 0.016*"industri" + 0.015*"face" + 0.015*"across" + 0.014*"password" + 0.014*"recognit" + 0.013*"revolution" + 0.013*"facial" + 0.013*"exampl"

# Score: 0.0340876542031765	 
# Topic: 0.040*"join" + 0.035*"compani" + 0.030*"build" + 0.026*"light" + 0.026*"sever" + 0.026*"blue" + 0.025*"pacif" + 0.025*"northwest" + 0.025*"tribut" + 0.011*"blockchain"

# Score: 0.03353748843073845	 
# Topic: 0.019*"great" + 0.018*"microsoft" + 0.015*"thank" + 0.012*"translat" + 0.012*"look" + 0.011*"team" + 0.011*"like" + 0.009*"take" + 0.009*"work" + 0.009*"artificialintellig"

# Score: 0.03333783522248268	 
# Topic: 0.210*"digit" + 0.108*"need" + 0.106*"learn" + 0.104*"peac" + 0.103*"world" + 0.101*"togeth" + 0.100*"stopcyberwarfar" + 0.098*"bethegener" + 0.003*"work" + 0.003*"everyth"


dict_py = {}

cols = []
for i in range(0, lda_model.num_topics):
    cols.append("Topic"+str(i))
print(cols)
val = []
for i in range(0,len(tweet_dictionary)):
    bow = dictionary.doc2bow(tweet_dictionary[i].split())
    t = lda_model.get_document_topics(bow)
    arr = []
    for ind in range(len(t)):
        arr.append(t[ind][1])
    val.append(arr)
df1 = pd.DataFrame(val, columns=cols)

### Writing to outout file ###
df1
df = pd.concat([df.reset_index(drop=True),df1.reset_index(drop=True)], axis=1)
df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.to_csv(path+"Topic.csv")

# df.head(10)
# result = pd.concat([df,df1], axis=0, join='outer', ignore_index=False)
# result

##for graph##
# import pyLDAvis.gensim
# pyLDAvis.enable_notebook()
# pyLDAvis.gensim.prepare(lda_model, corpus_bow, dictionary)


In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))